# Information extraction using tf-idf


In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

loading in readme file and making them in to tokens

In [2]:
from os import listdir
from os.path import isfile, join

contents = []
PATH = "../data"
files = [f"{PATH}/{f}" for f in listdir(PATH) if isfile(join(PATH, f))]

for file_path in files: 
    with open(file_path, "r") as f:
        contents.append(f.read())

contents[0][:50]

'# [React](https://reactjs.org/) &middot; [![GitHub'

In [3]:

def load_stop_words(path:str="../stopwords.txt"):
   words = []
   with open(path, "r") as f: 
      words = f.read().split("\n")
   assert len(words) != 0, "no stop words were found"
   return words

In [4]:



def clean(content:list):
    ascii_char = [chr(i) for i in range(0,255)]
    numbers = "0123456789"
    non_acc_char =  "\n,.()[]`/:-_*=\\<>|&%@?!\"\'#" + numbers
    non_acc_tokens = ["https","www", "com", "org", "license"]
    stop_words = load_stop_words()
    for i, _ in enumerate(content):
        for c in non_acc_char:
            content[i] = content[i].replace(c, " ")
        content[i] = content[i].split(" ")
        content[i] = list(filter(lambda c: c != "", content[i]))
        content[i] = [t for t in content[i] if not t in non_acc_tokens ] 
        content[i] = [s.lower() for s in content[i] if all(c in ascii_char for c in s)]
        content[i] = [t for t in content[i] if not t in stop_words] 
    return [" ".join(con) for con in content]


contents = clean(contents)
contents

['react reactjs middot; github img shields io badge mit blue svg github facebook react blob license npm version img shields io npm react svg style flat npmjs package react circleci status circleci gh facebook react svg style shield circleci gh facebook react prs welcome img shields io badge prs welcome brightgreen svg reactjs docs contribute html first pull request react javascript library building user interfaces declarative react makes painless create interactive uis design simple views state application react will efficiently update render just right components data changes declarative views code predictable simpler understand easier debug component based build encapsulated components manage state compose complex uis component logic written javascript templates can easily pass rich data app keep state dom learn once write don assumptions rest technology stack can develop new features react rewriting existing code react can render server using node power mobile apps using react nativ

In [5]:
def TfIdf(content:list):
    vectorizer = TfidfVectorizer()
    vecs = vectorizer.fit_transform(content)
    feature_names = vectorizer.get_feature_names_out()
    dense = vecs.todense()
    dense_list = dense.tolist()
    df = pd.DataFrame(dense_list, columns=feature_names)
    return df

tf_idf_data = TfIdf(contents)


In [6]:
def query_data(tf_idf_data:dict, query:str, result_size:int=2)->list:
    query = clean([query])[0]
    result = []
    words_tf_idf = {}
    query = query.lower()
    query_words = query.split(" ")
    for word in query_words: 
        if word in tf_idf_data.keys():
            words_tf_idf[word] = tf_idf_data[word]
    highest = 0
    for word, tf_idf in words_tf_idf.items():
        for idx, val in tf_idf.items(): 
            if val > highest: 
                result = idx
    return [result]



In [7]:
data = tf_idf_data.to_dict()
query = "I want to build web apps "
idxs = query_data(data, query)
for idx in idxs:
    print(files[idx])

../data/react_README.md
